In [1]:
import pandas as pd
import scipy as sp
import seaborn as sns
import numpy as np

In [2]:
data = pd.read_excel('ZIV_19.xlsx', sheet_name='Данные')
pd.set_option('display.max_rows', None)
print(data)

        X     Y   C         D       E    F      G    H    I
0   110.7  59.6 NaN  Задача 1     NaN  NaN    NaN  NaN  NaN
1   106.3  68.4 NaN       X0=  108.65  dx=  12.18   r=  3.0
2   108.7  72.7 NaN       Y0=   60.15  dy=   4.91   s=  5.0
3   118.2  74.8 NaN     alfa=    0.05  NaN    NaN  NaN  NaN
4   109.1  79.2 NaN       NaN     NaN  NaN    NaN  NaN  NaN
5   100.5  63.6 NaN  Задача 2     NaN  NaN    NaN  NaN  NaN
6   105.1  71.7 NaN  Y при X=  114.00  NaN    NaN  NaN  NaN
7   124.9  70.9 NaN       NaN     NaN  NaN    NaN  NaN  NaN
8   120.5  74.0 NaN  Задача 3     NaN  NaN    NaN  NaN  NaN
9   110.6  61.2 NaN     alfa=    0.10  NaN    NaN  NaN  NaN
10  118.5  67.7 NaN       NaN     NaN  NaN    NaN  NaN  NaN
11  100.9  70.5 NaN       NaN     NaN  NaN    NaN  NaN  NaN
12  117.5  73.0 NaN       NaN     NaN  NaN    NaN  NaN  NaN
13  102.4  62.8 NaN       NaN     NaN  NaN    NaN  NaN  NaN
14  114.5  73.5 NaN       NaN     NaN  NaN    NaN  NaN  NaN
15  106.7  66.8 NaN       NaN     NaN  N

C:\Users\Vitaly\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
x, y = list(data['X']), list(data['Y'])

x0 = data.at[1, 'E']
dx = data.at[1, 'G']
r = int(data.at[1, 'I'])

y0 = data.at[2, 'E']
dy = data.at[2, 'G']
s = int(data.at[2, 'I'])

alpha = data.at[3, 'E']

n = len(data)

In [4]:
interval1 = [x0 + i*dx for i in range(r-1)] + [float('inf')]
interval2 = [y0 + i*dy for i in range(s-1)] + [float('inf')]

In [5]:
data_vec = list(zip(x, y))
freq = [[
            len([1
            for vec in data_vec
            if (0 < i <= r and interval1[i-1] < vec[0] < interval1[i] or i == 0 and vec[0] < interval1[0])
            and (0 < j <= s and interval2[j-1] < vec[1] < interval2[j] or j == 0 and vec[1] < interval2[0])
        ])
        for i in range(r)
    ]
    for j in range(s)
]
# freq.reverse()
print(np.array(freq))
freq_df = pd.DataFrame(freq)
freq_df.columns = ['< ' + str(i1) for i1 in interval1]
freq_df.index = ['< ' + str(i2) for i2 in interval2]
print(freq_df.transpose().iloc[::-1])


[[ 0  8 10]
 [ 8  5  5]
 [ 7  4  7]
 [ 8 10  4]
 [ 6  2  1]]
                      < 60.15  < 65.06  < 69.97  < 74.88  < inf
< inf                      10        5        7        4      1
< 120.82999999999998        8        5        4       10      2
< 108.64999999999999        0        8        7        8      6


In [7]:
etasums = [sum(freq[i]) for i in range(s)]
freq_t = list(zip(*freq))
xisums = [sum(freq_t[i]) for i in range(r)]
print('Суммы для каждого интервала eta', etasums)
print('Суммы для каждого интервала xi', xisums)
overallsums = sum(etasums)
print('Всего', overallsums)

Суммы для каждого интервала eta [18, 18, 18, 22, 9]
Суммы для каждого интервала xi [29, 29, 27]
Всего 85


In [8]:
# Статистика Xi2
chi2 = sum(sum(
        (n*freq_df.iat[k, j] - xisums[j]*etasums[k])**2 / (xisums[j]*etasums[k])
        for k in range(s))
    for j in range(r)) / n
print('Статистика:', chi2)

Статистика: 18.445889288801165


In [9]:
# Проверка гипотез
Ccrit = sp.stats.chi2.ppf(1-alpha, (r-1)*(s-1))
print('C критическая:', Ccrit)
pval = 1 - sp.stats.chi2.cdf(chi2, (r-1)*(s-1))
print('p-value', pval)
print('Степени свободы:', (r-1) * (s-1))

assert (chi2 >= Ccrit) == (pval <= alpha)
if chi2 >= Ccrit:
    print('Нулевая гипотеза отвергается')
else:
    print('Нулевая гипотеза принимается')

C критическая: 15.507313055865453
p-value 0.018121059043094467
Степени свободы: 8
Нулевая гипотеза отвергается
